# Run a classifer on the branchwater search output

In [1]:
# base checkout of workflow directory is here:
BASE='/home/ctbrown/scratch3/2025-workflow-core99/'

# parquet files from branchwater
BASE_OUTPUTS=BASE+'/outputs.branchwater'

In [2]:
import polars as pl
import numpy as np
import sklearn.tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import balanced_accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

In [3]:
dirpath = BASE_OUTPUTS+'/*.parquet'

bw_df = pl.scan_parquet(dirpath).collect()
bw_df = bw_df.with_columns(pl.col("query_name").alias("species"),
                           pl.col("match_name").alias("acc"))
bw_df = bw_df.select(["species", "acc", "containment", "intersect_hashes"])

# mimic the Web site search threshold of 0.1
bw_df = bw_df.filter(pl.col("containment") >= .1)
bw_df

species,acc,containment,intersect_hashes
str,str,f64,i64
"""s__Bariatricus sp004560705""","""SRR9866666""",0.689906,15002
"""s__Bariatricus sp004560705""","""SRR8960383""",0.681168,14812
"""s__Bariatricus sp004560705""","""SRR17241623""",0.677121,14724
"""s__Bariatricus sp004560705""","""ERR2241669""",0.677029,14722
"""s__Bariatricus sp004560705""","""SRR11185229""",0.676707,14715
…,…,…,…
"""s__UBA2868 sp004552595""","""SRR11183402""",0.100044,2263
"""s__UBA2868 sp004552595""","""SRR20591093""",0.100044,2263
"""s__UBA2868 sp004552595""","""SRR11125061""",0.1,2262


In [4]:
metadata_df = pl.scan_parquet('/group/ctbrowngrp5/sra-metagenomes/20241128-metadata.parquet')
metadata_df = metadata_df.select(["acc", "organism"]).filter(pl.col("acc") != "NP")
metadata_df = metadata_df.collect()
metadata_df

acc,organism
str,str
"""SRR28523869""","""human metagenome"""
"""SRR19901137""","""Streptococcus suis"""
"""SRR24962475""","""Escherichia coli"""
"""SRR29161330""","""biofilm metagenome"""
"""SRR26668739""","""bovine gut metagenome"""
…,…
"""ERR2709388""","""human gut metagenome"""
"""SRR089825""","""Neurospora crassa"""
"""SRR14870612""","""Frankiales bacterium"""


In [5]:
bw_df = bw_df.join(metadata_df, on="acc", how="left")
bw_df

species,acc,containment,intersect_hashes,organism
str,str,f64,i64,str
"""s__Bariatricus sp004560705""","""SRR9866666""",0.689906,15002,"""Lawsonia intracellularis"""
"""s__Bariatricus sp004560705""","""SRR8960383""",0.681168,14812,"""pig gut metagenome"""
"""s__Bariatricus sp004560705""","""SRR17241623""",0.677121,14724,"""pig gut metagenome"""
"""s__Bariatricus sp004560705""","""ERR2241669""",0.677029,14722,"""metagenome"""
"""s__Bariatricus sp004560705""","""SRR11185229""",0.676707,14715,"""pig gut metagenome"""
…,…,…,…,…
"""s__UBA2868 sp004552595""","""SRR11183402""",0.100044,2263,"""pig gut metagenome"""
"""s__UBA2868 sp004552595""","""SRR20591093""",0.100044,2263,"""pig gut metagenome"""
"""s__UBA2868 sp004552595""","""SRR11125061""",0.1,2262,"""pig gut metagenome"""


In [6]:
bw_df['organism'].value_counts().sort(by='count', descending=True)

organism,count
str,u32
"""human gut metagenome""",346274
"""metagenome""",71920
"""pig gut metagenome""",66370
"""gut metagenome""",65308
"""Homo sapiens""",28321
…,…
"""human milk metagenome""",1
"""Parabacteroides distasonis""",1
"""Eubacterium sp. 3_1_31""",1


In [7]:
rename_df = []
for organism in bw_df['organism'].unique().to_list():
    simpleorg = 'unknown'
    if organism:
        organism = organism.lower()
        for kw in ['human', 'homo']:
            if kw in organism.lower():
                simpleorg = 'human'
                break
        for kw in ['pig', 'sus', 'scrofa']:
            if kw in organism.lower():
                simpleorg = 'pig'
                break
    rename_df.append(dict(organism=organism, simpleorg=simpleorg))

rename_df = pl.DataFrame(rename_df)
rename_df['simpleorg'].value_counts().sort(by='count', descending=True)


simpleorg,count
str,u32
"""unknown""",255
"""human""",13
"""pig""",6


In [8]:
rename_df

organism,simpleorg
str,str
"""coprococcus""","""unknown"""
"""skin metagenome""","""unknown"""
"""biofilm metagenome""","""unknown"""
"""bioleaching metagenome""","""unknown"""
"""faecalibacterium prausnitzii""","""unknown"""
…,…
"""escherichia""","""unknown"""
"""methanobrevibacter smithii""","""unknown"""
"""human bile metagenome""","""human"""


In [9]:
# add a new column with a simplified organism
bw_df = bw_df.join(rename_df, on='organism', how='inner', coalesce=True).select(['acc', 'organism', 'species', 'simpleorg'])
bw_df.select(['acc', 'simpleorg']).unique()['simpleorg'].value_counts()


simpleorg,count
str,u32
"""human""",129493
"""unknown""",66933
"""pig""",6569


In [10]:
bw_df

acc,organism,species,simpleorg
str,str,str,str
"""SRR8960383""","""pig gut metagenome""","""s__Bariatricus sp004560705""","""pig"""
"""SRR17241623""","""pig gut metagenome""","""s__Bariatricus sp004560705""","""pig"""
"""ERR2241669""","""metagenome""","""s__Bariatricus sp004560705""","""unknown"""
"""SRR11185229""","""pig gut metagenome""","""s__Bariatricus sp004560705""","""pig"""
"""ERR2241674""","""metagenome""","""s__Bariatricus sp004560705""","""unknown"""
…,…,…,…
"""SRR11183402""","""pig gut metagenome""","""s__UBA2868 sp004552595""","""pig"""
"""SRR20591093""","""pig gut metagenome""","""s__UBA2868 sp004552595""","""pig"""
"""SRR11125061""","""pig gut metagenome""","""s__UBA2868 sp004552595""","""pig"""


In [11]:
def make_matrices(df):
    acc_df = df['acc'].unique().to_frame().with_row_index(name='acc_index')
    species_df = df['species'].unique().to_frame().with_row_index(name='species_index')
    org_df = df['simpleorg'].unique().to_frame().with_row_index(name='org_index')

    df = df.join(org_df, on='simpleorg', how='left')
    df = df.join(acc_df, on='acc', how='left')
    df = df.join(species_df, on='species', how='left')

    obs = np.zeros((len(acc_df), len(species_df)))
    target = np.zeros((len(acc_df)))

    for row in df.iter_rows(named=True):
        acc_id = row["acc_index"]
        org_id = row["org_index"]
        species_id = row["species_index"]

        obs[acc_id, species_id] = 1
        target[acc_id] = org_id

    print(f'observations matrix shape is: {obs.shape}')

    return df, obs, target


In [12]:
human_pig_only = bw_df.filter(pl.col("simpleorg") != "unknown")
hp_df, hp_obs, hp_target = make_matrices(human_pig_only)

observations matrix shape is: (136062, 16)


In [13]:
hp_df

acc,organism,species,simpleorg,org_index,acc_index,species_index
str,str,str,str,u32,u32,u32
"""SRR8960383""","""pig gut metagenome""","""s__Bariatricus sp004560705""","""pig""",0,42084,6
"""SRR17241623""","""pig gut metagenome""","""s__Bariatricus sp004560705""","""pig""",0,41356,6
"""SRR11185229""","""pig gut metagenome""","""s__Bariatricus sp004560705""","""pig""",0,4046,6
"""SRR17241692""","""pig gut metagenome""","""s__Bariatricus sp004560705""","""pig""",0,60067,6
"""SRR17241693""","""pig gut metagenome""","""s__Bariatricus sp004560705""","""pig""",0,9845,6
…,…,…,…,…,…,…
"""SRR11183337""","""pig gut metagenome""","""s__UBA2868 sp004552595""","""pig""",0,48851,4
"""SRR11126455""","""pig gut metagenome""","""s__UBA2868 sp004552595""","""pig""",0,121177,4
"""SRR11183402""","""pig gut metagenome""","""s__UBA2868 sp004552595""","""pig""",0,60473,4


## 6-Fold Cross Validation approach

Run 6 different splits of the data; train on 1/6 of the data and test on the other 5/6s of the data, and then repeat that for each of the 6 splits.

In [14]:
kf = StratifiedKFold(n_splits=6)

accuracies = []
i = 1
for train_sub, test_sub in kf.split(hp_obs, hp_target):
    dt = DecisionTreeClassifier(random_state=42)
    tree = dt.fit(hp_obs[train_sub], hp_target[train_sub])
    pred = tree.predict(hp_obs[test_sub])

    accuracy = balanced_accuracy_score(hp_target[test_sub], pred)
    accuracies.append(accuracy)
    print(f"iteration {i}: accuracy {accuracy:.3f}")
    i += 1

print(f"mean accuracy across {i-1} splits: {np.mean(accuracies):.3f}")

iteration 1: accuracy 0.986
iteration 2: accuracy 0.993
iteration 3: accuracy 0.988
iteration 4: accuracy 0.990
iteration 5: accuracy 0.990
iteration 6: accuracy 0.990
mean accuracy across 6 splits: 0.989


## What do we after 6-fold cross validation?

This approach lets us check to see that, in general, the performance of any classifer trained on this data will be pretty good - independent of which set of data we use.

Now, to train the best possible classifier for _future_ use, we will use all the data to train:

In [15]:
dt = DecisionTreeClassifier(random_state=42)
tree = dt.fit(hp_obs, hp_target)
pred = tree.predict(hp_obs)

accuracy = balanced_accuracy_score(hp_target, pred)
print(f"full classifier: accuracy {accuracy:.3f}")

full classifier: accuracy 0.992


## What do we do with the full classifier now??

Now we can go back to the original data - not just the human/pig subset, but the one with unknowns. (Or, potentially, new data sets.)

The only thing we need to be careful about is to encode new observations using the species indexes that match the trained classifier, as well as the organism indexes (0 pig, 1 human). Let's extract those:

In [16]:
species_index_df = hp_df.select(['species_index', 'species']).unique()
species_index_df

species_index,species
u32,str
0,"""s__Holdemanella porci"""
7,"""s__Prevotella sp002251295"""
10,"""s__JALFVM01 sp022787145"""
3,"""s__Fimisoma sp002320005"""
5,"""s__Lactobacillus amylovorus"""
…,…
9,"""s__Prevotella sp000434975"""
8,"""s__Roseburia inulinivorans"""
12,"""s__Cryptobacteroides sp9005469…"


In [17]:
org_df = hp_df.select(['org_index', 'simpleorg']).unique()
org_df

org_index,simpleorg
u32,str
0,"""pig"""
1,"""human"""


Now, let's take some of some of the data we didn't touch from the original bw_df, and encode and classify it: 

In [18]:
def make_obs_matrix(df, species_df):
    # need a new access mapping
    acc_df = df['acc'].unique().to_frame().with_row_index(name='acc_index')

    # construct new internal data frame, populate
    df = df.join(acc_df, on='acc', how='left')
    df = df.join(species_df, on='species', how='left')

    obs = np.zeros((len(acc_df), len(species_df)))

    for row in df.iter_rows(named=True):
        acc_id = row["acc_index"]
        species_id = row["species_index"]

        obs[acc_id, species_id] = 1

    print(f'observations matrix shape is: {obs.shape}')

    return df, obs


In [19]:
unknown_only_df = bw_df.filter(pl.col("simpleorg") == "unknown")
unk_df, unk_obs = make_obs_matrix(unknown_only_df, species_index_df)

observations matrix shape is: (66933, 16)


In [20]:
unknown_pred = tree.predict(unk_obs)
unknown_pred

array([1., 1., 1., ..., 1., 1., 1.], shape=(66933,))

## Predictions are great and all but...

Great! 67,000 predictions 😭 . What do we do with them??

Let's merge the predictions back into the spreadsheet.

In [21]:
unk_pred_df = []
for (row_n, predicted_org) in enumerate(unknown_pred):
    unk_pred_df.append(dict(acc_index=row_n, org_index=int(predicted_org)))
unk_pred_df = pl.DataFrame(unk_pred_df).join(org_df, on='org_index', how='left')

unk_acc_df = unk_df.select(["acc", "acc_index"]).unique()

unk_pred_df = unk_pred_df.join(unk_acc_df, on='acc_index', how='left')
unk_pred_df

acc_index,org_index,simpleorg,acc
i64,i64,str,str
0,1,"""human""","""SRR15730559"""
1,1,"""human""","""SRR25336626"""
2,1,"""human""","""SRR21858605"""
3,1,"""human""","""ERR720132"""
4,1,"""human""","""ERR8610074"""
…,…,…,…
66928,1,"""human""","""DRR585793"""
66929,1,"""human""","""SRR6122151"""
66930,1,"""human""","""ERR4333931"""


In [22]:
unk_pred_df["simpleorg"].value_counts().sort(by='count', descending=True)

simpleorg,count
str,u32
"""human""",60548
"""pig""",6385


## OK! We have our predictions... did this actually work?

In [23]:
# sample 5 randomly
unk_pred_df.sample(n=5)

acc_index,org_index,simpleorg,acc
i64,i64,str,str
7925,1,"""human""","""SRR30480317"""
30462,1,"""human""","""ERR2227914"""
44988,1,"""human""","""SRR27675056"""
2738,0,"""pig""","""SRR9218182"""
5477,0,"""pig""","""ERR3393542"""


In [24]:
# sample 5 predicted as pig
unk_pred_df.filter(pl.col("simpleorg") == "pig").sample(n=5)

acc_index,org_index,simpleorg,acc
i64,i64,str,str
57490,0,"""pig""","""SRR18352865"""
4071,0,"""pig""","""SRR23203996"""
6839,0,"""pig""","""SRR26458527"""
41541,0,"""pig""","""SRR15970567"""
61390,0,"""pig""","""SRR22560192"""


In [25]:
# sample 5 predicted as human
unk_pred_df.filter(pl.col("simpleorg") == "human").sample(n=5)

acc_index,org_index,simpleorg,acc
i64,i64,str,str
6092,1,"""human""","""ERR3573540"""
35681,1,"""human""","""SRR21938720"""
5337,1,"""human""","""ERR4320075"""
61410,1,"""human""","""ERR10024943"""
58346,1,"""human""","""SRR27419192"""


# What features are most important for good classification?

First, let's get the feature importances.

In [26]:
importances_df = []
for (species_index, importance) in enumerate(tree.feature_importances_):
    d = dict(species_index=species_index, importance=importance)
    importances_df.append(d)

importances_df = pl.DataFrame(importances_df)
importances_df = importances_df.join(species_index_df, on='species_index', how='left')
importances_df = importances_df.sort(by='importance', descending=True)
importances_df

species_index,importance,species
i64,f64,str
5,0.795885,"""s__Lactobacillus amylovorus"""
4,0.066051,"""s__UBA2868 sp004552595"""
8,0.052154,"""s__Roseburia inulinivorans"""
13,0.024844,"""s__Gemmiger qucibialis"""
3,0.018973,"""s__Fimisoma sp002320005"""
…,…,…
9,0.001556,"""s__Prevotella sp000434975"""
11,0.00128,"""s__Phascolarctobacterium_A suc…"
0,0.001194,"""s__Holdemanella porci"""


Now let's rebuild the classifier after removing each species, and see how well it does...

In [27]:
species_in_order = importances_df['species'].to_list()
species_in_order

['s__Lactobacillus amylovorus',
 's__UBA2868 sp004552595',
 's__Roseburia inulinivorans',
 's__Gemmiger qucibialis',
 's__Fimisoma sp002320005',
 's__Mogibacterium_A kristiansenii',
 's__JAFBIX01 sp021531895',
 's__Prevotella sp002251295',
 's__Sodaliphilus sp004557565',
 's__Phocaeicola vulgatus',
 's__Bariatricus sp004560705',
 's__Prevotella sp000434975',
 's__Phascolarctobacterium_A succinatutens',
 's__Holdemanella porci',
 's__JALFVM01 sp022787145',
 's__Cryptobacteroides sp900546925']

In [28]:
human_pig_only = bw_df.filter(pl.col("simpleorg") != "unknown")
species_to_rm = list(species_in_order)

remove_df = human_pig_only
while len(species_to_rm) >= 2:
    rm_species = species_to_rm.pop(0)
    print(f'removing {rm_species} too')

    remove_df = remove_df.filter(pl.col("species") != rm_species)
    augmented_df, rm_obs, rm_target = make_matrices(remove_df)

    dt = DecisionTreeClassifier(random_state=42)
    tree = dt.fit(rm_obs, rm_target)
    pred = tree.predict(rm_obs)

    accuracy = balanced_accuracy_score(rm_target, pred)
    print(f"classifier accuracy with {remove_df['species'].n_unique()} is: {accuracy:.3f}")
    print('---')

removing s__Lactobacillus amylovorus too
observations matrix shape is: (135927, 15)
classifier accuracy with 15 is: 0.989
---
removing s__UBA2868 sp004552595 too
observations matrix shape is: (135916, 14)
classifier accuracy with 14 is: 0.990
---
removing s__Roseburia inulinivorans too
observations matrix shape is: (130619, 13)
classifier accuracy with 13 is: 0.985
---
removing s__Gemmiger qucibialis too
observations matrix shape is: (120317, 12)
classifier accuracy with 12 is: 0.974
---
removing s__Fimisoma sp002320005 too
observations matrix shape is: (120267, 11)
classifier accuracy with 11 is: 0.962
---
removing s__Mogibacterium_A kristiansenii too
observations matrix shape is: (119976, 10)
classifier accuracy with 10 is: 0.962
---
removing s__JAFBIX01 sp021531895 too
observations matrix shape is: (119876, 9)
classifier accuracy with 9 is: 0.928
---
removing s__Prevotella sp002251295 too
observations matrix shape is: (119803, 8)
classifier accuracy with 8 is: 0.918
---
removing s__